# Exploring Address Parsing APIs

## 0. Setup

I considered the following APIs:
* scourgify
* usaddress
* postal_address (poor documentation)
* address (out of date- api uses prints intended for Python 2)

After looking at docs and testing the implementation of each library, I narrowed down the list to two:
* scourgify
* usaddress

### Imports

In [1]:
import usaddress # usaddress
import scourgify # usaddress-scourgify

### Set Address

In [2]:
address = '5137 Pond Crest Trail, Fairview, TX 75069'

## 1. Scourgify

Follows RESO guidelines and USPS pub 28. 

#### Notes:
* Scourgify does not attempt to validate an address

In [3]:
# Returns a dictionary of "normalized" address data, but does not validate
scourgify_result = scourgify.normalize_address_record(address)

print("scourgify result: \n")
scourgify_result

scourgify result: 



{'address_line_1': '5137 POND CREST TRL',
 'address_line_2': None,
 'city': 'FAIRVIEW',
 'state': 'TX',
 'postal_code': '75069'}

## 2. USAddress

usaddress is a Python library for parsing unstructured address strings into address components, using advanced NLP methods.

#### Notes: 
* Probabilistic parsing
* Does not validate address

In [4]:
# Returns an array of 
usaddress_result = usaddress.parse(address)

print("usaddress result: \n")
usaddress_result

usaddress result: 



[('5137', 'AddressNumber'),
 ('Pond', 'StreetName'),
 ('Crest', 'StreetName'),
 ('Trail,', 'StreetNamePostType'),
 ('Fairview,', 'PlaceName'),
 ('TX', 'StateName'),
 ('75069', 'ZipCode')]